In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import wrangle as w
from env import user, password, host

### 1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [ ]:
url = f"mysql+pymysql://{user}:{password}@{host}/zillow"

In [ ]:
query = '''
            
SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
FROM properties_2017

LEFT JOIN propertylandusetype USING(propertylandusetypeid)

WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")'''

In [ ]:
def get_zillow_data():
    ''' Retrieve data from Zillow database within codeup, selecting specific features 
    '''
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        query = '''
            
        SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        FROM properties_2017

        LEFT JOIN propertylandusetype USING(propertylandusetypeid)

        WHERE propertylandusedesc IN ("Single Family Residential",                       
                              "Inferred Single Family Residential")'''
        df = pd.read_sql(query, get_connection('zillow'))

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df
    
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [2]:
df = w.get_zillow_data()

In [3]:
df.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


### 2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.info(null_counts=True)

In [ ]:
# Find the total number of Null values in each column of our DataFrame.
df.isnull().sum()

In [ ]:
# Check for any Null values in each column of our DataFrame.
df.isnull().any()

In [ ]:
# Return the names for any columns in our DataFrame with any Null values.
df.columns[df.isnull().any()]

In [ ]:
#Building a function to clean the data

def clean_zillow(df):
    ''' This function takes in zillow data and returns a clean dataset.
    '''
    # renaming columns
    df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms', 
                          'calculatedfinishedsquarefeet':'area',
                          'taxvaluedollarcnt':'tax_value', 
                          'yearbuilt':'year_built',
                          'taxamount':'tax_amount'
                    })
    
    # Replace a whitespace sequence or empty with a NaN value and reassign this manipulation to df. 
    df = df.replace(r'^\s*$', np.nan, regex=True)
    

In [4]:
#Calling this function from my wrangle file
w.clean_zillow(df)

In [ ]:
# renaming columns
df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms', 
                          'calculatedfinishedsquarefeet':'area',
                          'taxvaluedollarcnt':'tax_value', 
                          'yearbuilt':'year_built',
                          'taxamount':'tax_amount'
                    })

In [5]:
df.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [ ]:
# Replace a whitespace sequence or empty with a NaN value and reassign this manipulation to df.

df = df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# making distribution plots BEDROOMS

sns.displot(x='bedrooms', data=df, bins=5)
plt.title('bedrooms')
plt.show()

In [ ]:
sns.displot(x='bathrooms', data=df, bins=5)
plt.title('bathrooms')
plt.show()

In [ ]:
sns.displot(x='area', data=df, bins=5)
plt.title('area')
plt.show()

In [ ]:
sns.displot(x='year_built', data=df, bins=5)
plt.title('year_built')
plt.show()

In [ ]:
sns.displot(x='tax_value', data=df, bins=5)
plt.title('tax_value')
plt.show()

In [ ]:
sns.displot(x='tax_amount', data=df, bins=5)
plt.title('tax_amount')
plt.show()

In [ ]:
#Looking at outliers
# List of columns
cols = ['bedrooms', 'bathrooms', 'area', 'tax_value', 'tax_amount']

for i, col in enumerate(cols):

    plot_number = i + 1
    # Create subplot
    plt.subplot(1, len(cols), plot_number)
    # Title with column name
    plt.title(col)
    # Display boxplot for column.
    sns.boxplot(data=df[[col]])
    # sets proper spacing between plots
    plt.tight_layout()

plt.show()

In [ ]:
#Taking IQR approach to removing outliers
# IQR
Q1 = np.percentile(df['bedrooms'], 25,
				interpolation = 'midpoint')

Q3 = np.percentile(df['bedrooms'], 75,
				interpolation = 'midpoint')
IQR = Q3 - Q1

In [ ]:
def outliers(df, feature):
    Q1= df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR
    return upper_limit, lower_limit

In [ ]:
#IQR range for bedrooms
upper, lower = outliers(df, "bedrooms")
print("Upper whisker: ", upper)
print("Lower Whisker: ", lower)

In [ ]:
#IQR range for bathrooms
upper,lower = outliers(df,'bathrooms')
print("Upper whisker: ", upper)
print("Lower Whisker: ", lower)

In [ ]:
#IQR range for area
upper,lower = outliers(df,'area')
print("Upper whisker: ", upper)
print("Lower Whisker: ", lower)

In [ ]:
df.info()

In [ ]:
#Making a function to remove the outliers from entire dataset
#Using 1.5 for limits

def remove_outliers(df,feature_list):
    
    for feature in feature_list:
    
        #define interquartile range
        Q1= df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        #Set limits
        upper_limit = Q3 + 1.5 * IQR
        lower_limit = Q1 - 1.5 * IQR
        #remove outliers
        df = df[(df[feature] > lower_limit) & (df[feature] < upper_limit)]
    
    return df

In [ ]:
df = remove_outliers(df,['bedrooms','bathrooms','area','tax_value','tax_amount'])

In [ ]:
df.info()

In [ ]:
#Checking for null values now
df.isnull().sum()

In [ ]:
df.info(null_counts=True)

In [6]:
#Remove remainder of outliers
df.dropna(inplace = True)

In [7]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152863
Data columns (total 8 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Unnamed: 0                    2140235 non-null  int64  
 1   bedroomcnt                    2140235 non-null  float64
 2   bathroomcnt                   2140235 non-null  float64
 3   calculatedfinishedsquarefeet  2140235 non-null  float64
 4   taxvaluedollarcnt             2140235 non-null  float64
 5   yearbuilt                     2140235 non-null  float64
 6   taxamount                     2140235 non-null  float64
 7   fips                          2140235 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 147.0 MB


In [8]:
df.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
